In [1]:
print("hello")

hello


In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [9]:
from langchain_groq import ChatGroq

### now we define model and temperature tokens etc   
also trying to invoke the model

In [ ]:

llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

# 3. Test the connection
try:
    response = llm.invoke("Hello Groq! Give me a 5-word summary of what an LLM is.")
    print("Success! Response:")
    print(response.content)
except Exception as e:
    print(f"Error: {e}")
    print("Check if your GROQ_API_KEY is correct in the .env file.")

Success! Response:
Large AI language model.


now we invoke the model

In [17]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime la programmation.", additional_kwargs={'reasoning_content': 'Okay, the user wants to translate "I love programming." into French. Let me start by breaking down the sentence. "I love" is straightforward. In French, "I love" is "J\'aime". Now, "programming" can be a bit tricky. The direct translation is "la programmation", but sometimes in French, they might use "le programmage" depending on the context. However, "programmation" is more commonly used in the context of computer programming. So "J\'aime la programmation." That should be correct. Let me double-check to make sure there\'s no regional variation or alternative phrasing. Yeah, "programmation" is the standard term here. So the translation is accurate.\n'}, response_metadata={'token_usage': {'completion_tokens': 162, 'prompt_tokens': 33, 'total_tokens': 195, 'completion_time': 0.296414774, 'completion_tokens_details': {'reasoning_tokens': 150}, 'prompt_time': 0.001168041, 'prompt_tokens_details': None, '

**Basic Agent**    
decide which tools to use, and iteratively work towards solutions.  

Core components  
​
Model  
The model is the reasoning engine of your agent. It can be specified in multiple ways, supporting both static and dynamic model selection.  
​
Static model  
Static models are configured once when creating the agent and remain unchanged throughout execution. This is the most common and straightforward approach.

In [56]:
from langchain.agents import create_agent
tools={}
agent1 = create_agent(llm,tools=tools) #
result = agent1.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]}
)
result # we did not provide any tool

{'messages': [HumanMessage(content="What's the weather in San Francisco?", additional_kwargs={}, response_metadata={}, id='18727a70-1c3f-4ef2-ac25-12bde5cfbf36'),
  AIMessage(content='I can\'t provide real-time weather updates, but here’s how you can check the current weather in San Francisco:\n\n1. **Weather Websites/Apps**: Use services like [Weather.com](https://www.weather.com), [AccuWeather](https://www.accuweather.com), or the [National Weather Service (NWS)](https://www.weather.gov) for accurate forecasts.\n2. **Search Engine**: Type "current weather San Francisco" into Google or Bing for an instant summary.\n3. **Smart Devices**: Ask a smart speaker (e.g., Alexa, Google Assistant) or check your phone’s built-in weather app.\n\n**Note**: San Francisco is known for foggy mornings, cool temperatures, and sudden weather changes. Coastal areas may be windier, while inland neighborhoods can be slightly warmer. Always check for the latest updates before heading out! 🌫️', additional_kw

**Dynamic model**  

Dynamic models are selected at runtime based on the current state and context. This enables sophisticated routing logic and cost optimization.  

To use a dynamic model, create middleware using the @wrap_model_call decorator that modifies the model in the request:

In [65]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = llm
advanced_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))

agent2 = create_agent(
    model=basic_model,
    middleware=[dynamic_model_selection]
)
result = agent2.invoke(
    {"messages": [{"role": "user", "content": "What's is llm"}]}
)
result

{'messages': [HumanMessage(content="What's is llm", additional_kwargs={}, response_metadata={}, id='92f11e32-3efe-4ab8-94f5-d8547611db5d'),
  AIMessage(content='**LLM** stands for **Large Language Model**, a type of artificial intelligence (AI) system designed to understand, generate, and interact with human language. These models are trained on vast amounts of text data (e.g., books, articles, websites) to learn patterns, grammar, context, and meaning, enabling them to perform tasks like answering questions, writing stories, translating languages, and more.\n\n### Key Features of LLMs:\n1. **Scale**: They use massive datasets and complex neural networks (often based on the **Transformer** architecture) to process and generate text.\n2. **Versatility**: They can handle a wide range of tasks, from simple queries to creative writing or coding.\n3. **Context Understanding**: They analyze input text to provide relevant, coherent responses based on the context provided.\n4. **Training**: Th

**Now We provide Tools**  
Tools give agents the ability to take actions. Agents go beyond simple model-only tool binding by facilitating:  
Multiple tool calls in sequence (triggered by a single prompt)  
Parallel tool calls when appropriate  
Dynamic tool selection based on previous results  
Tool retry logic and error handling  
State persistence across tool calls

In [70]:
from langchain.tools import tool
@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"

agent3 = create_agent(llm, tools=[search, get_weather])
result = agent3.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]}
)
result

{'messages': [HumanMessage(content="What's the weather in San Francisco?", additional_kwargs={}, response_metadata={}, id='29a0ba47-b442-48c3-b440-d4007d55c4d1'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in San Francisco. Let me check which tool I can use here. The available functions are search and get_weather. The get_weather function is specifically for getting weather information for a location. So I should use that. The parameters required are the location, which in this case is San Francisco. I need to make sure the arguments are correctly formatted as a JSON object. Let me structure the tool call with the name as get_weather and the arguments with location set to "San Francisco". That should do it.\n', 'tool_calls': [{'id': '4jw53ae4v', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens'

**System prompt**  
You can shape how your agent approaches tasks by providing a prompt. The system_prompt parameter can be provided as a string:

In [72]:
agent4 = create_agent(
    llm,
    tools,
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

result = agent4.invoke(
    {"messages": [{"role": "user", "content": "What's is llm"}]}
)
result

{'messages': [HumanMessage(content="What's is llm", additional_kwargs={}, response_metadata={}, id='81780b3d-07dc-486a-a7a2-a98676afb357'),
  AIMessage(content='An **LLM** (Large Language Model) is an AI system trained on vast amounts of text data to understand and generate human-like language. It can perform tasks like answering questions, writing stories, coding, translating, and more. Examples include models like **GPT**, **BERT**, and **LLaMA**. LLMs use deep learning to analyze patterns in language and generate coherent, context-aware responses. They are widely used in chatbots, content creation, and research.', additional_kwargs={'reasoning_content': 'Okay, the user is asking "What\'s is llm". First, I need to correct the typo. They probably meant "What is LLM?".\n\nLLM stands for Large Language Model. I should explain that it\'s a type of AI model trained on vast amounts of text data. These models can generate human-like text, answer questions, and perform various language tasks

**Structured output**  
In some situations, you may want the agent to return an output in a specific format.  
LangChain provides strategies for structured output via the response_format parameter.  

**ToolStrategy**  
ToolStrategy uses artificial tool calling to generate structured output. This works with any model that supports tool calling. ToolStrategy should be used when provider-native structured output (via ProviderStrategy) is not available or reliable.

In [107]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model=llm,
    tools=[search],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

  **ProviderStrategy** uses the model provider’s native structured output generation. This is more reliable but only works with providers that support native structured output:
  from langchain.agents.structured_output import ProviderStrategy  


agent = create_agent(
    model=llm,
    response_format=ProviderStrategy(ContactInfo)
)  
this was the original code but was not working so we needed to modify for grok model

In [96]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model=llm,
    response_format=ProviderStrategy(ContactInfo)
)
#Apply the ProviderStrategy
# This tells the model to use its native "Structured Output" logic
structured_model = model.with_structured_output(
    ContactInfo
)
result = structured_model.invoke(["Extract contact info from: John Doe, john@example.com, (555) 123-4567"]
)


result

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')


**Memory**  
Agents maintain conversation history automatically through the message state. You can also configure the agent to use a custom state schema to remember additional information during the conversation.  
Information stored in the state can be thought of as the short-term memory of the agent:  
Custom state schemas must extend AgentState as a TypedDict.  
There are two ways to define custom state:  
Via middleware (preferred)  
Via state_schema on create_agent

In [114]:
#Defining state via middleware  
from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware
from typing import Any


class CustomState(AgentState):
    user_preferences: dict

class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState
    tools = [search, get_weather]  # tool1 and tool2 converted to search and get_weather

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        ...

agent = create_agent(
    model=llm,
    tools=tools,
    middleware=[CustomMiddleware()]
)

# The agent can now track additional state beyond messages
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations on llm"}],# i added on llm
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})
result 

{'messages': [HumanMessage(content='I prefer technical explanations on llm', additional_kwargs={}, response_metadata={}, id='6fea86f5-c7ea-4fbc-8a68-ac7509379797'),
  AIMessage(content="I'll provide technical explanations on Large Language Models (LLMs). Let me know which specific aspects you'd like to explore - here are some technical areas I can explain:\n\n1. **Architecture** - Transformer-based designs, attention mechanisms, positional encoding\n2. **Training** - Supervised fine-tuning, reinforcement learning with human feedback (RLHF), loss functions\n3. **Optimization** - Gradient descent variants, learning rate scheduling, weight initialization\n4. **Scaling Laws** - How performance scales with model size, data quantity, and compute\n5. **Inference** - Beam search, temperature sampling, quantization for deployment\n6. **Efficiency** - Model compression, sparse architectures, memory optimization\n\nWould you like to dive into any of these technical topics specifically?", addition

**Defining state via state_schema**  
Use the state_schema parameter as a shortcut to define custom state that is only used in tools.

In [118]:
from langchain.agents import AgentState


class CustomState(AgentState):
    user_preferences: dict

agent = create_agent(
    model=llm,
    tools=[search, get_weather],
    state_schema=CustomState
)
# The agent can now track additional state beyond messages
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations on llm"}],# i added on llm
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})
result

{'messages': [HumanMessage(content='I prefer technical explanations on llm', additional_kwargs={}, response_metadata={}, id='20220aed-14f7-4566-96dc-ce63d49eadcb'),
  AIMessage(content="I'll provide a technical explanation of Large Language Models (LLMs). Would you like me to focus on a specific aspect such as:\n\n1. **Architecture** (transformers, attention mechanisms, etc.)\n2. **Training process** (pretraining, fine-tuning, loss functions)\n3. **Inference mechanics** (sampling strategies, beam search)\n4. **Optimization techniques** (weight pruning, quantization)\n5. **Evaluation metrics** (perplexity, BLEU scores)\n6. **Scaling laws and parameter efficiency**\n\nOr would you prefer a comprehensive overview?", additional_kwargs={'reasoning_content': 'Okay, the user mentioned they prefer technical explanations on LLMs. Let me break this down.\n\nFirst, I need to understand what exactly they\'re looking for. "Technical explanations" could cover a range of topics like architecture, tra

Defining custom state via middleware is preferred over defining it via state_schema on create_agent because it allows you to keep state extensions conceptually scoped to the relevant middleware and tools.  
state_schema is still supported for backwards compatibility on create_agent.

A list of messages can be provided to a chat model to represent conversation history.   
Each message has a role that models use to indicate who sent the message in the conversation.

In [119]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

response = llm.invoke(conversation)
print(response)  # AIMessage("J'adore créer des applications.")

content="J'adore développer des applications." additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love building applications" into French. Let me start by recalling the previous interaction where they asked for "I love programming" which I translated as "J\'adore la programmation." Now, the new sentence is similar but uses "building applications" instead of "programming."\n\nFirst, I need to translate "I love" again. The user used "J\'adore" before, so I should stick with that for consistency. Next, "building applications." The verb "building" here is in the gerund form, which in French can be translated using the present participle or the infinitive. However, in this context, since it\'s part of the phrase "love building," the infinitive is more appropriate. \n\n"Building" as an infinitive would be "construire" or "édifier." But "building applications" in the context of software development is more commonly translated as "développer des applications" or "

**Batch**  
Batching a collection of independent requests to a model can significantly improve performance and reduce costs, as the processing can be done in parallel:

In [121]:
responses = llm.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response)

content="Parrots exhibit vibrant, colorful feathers due to a combination of evolutionary, ecological, and physiological factors. Here's a structured explanation of the primary reasons:\n\n1. **Sexual Selection and Mate Attraction**:\n   - **Mate Choice**: Bright colors often signal genetic fitness and health. In many parrot species, males with more vivid plumage are preferred by females, as these colors can indicate a robust immune system and good foraging ability.\n   - **Health Indicators**: The intensity of colors, particularly those derived from carotenoid pigments (obtained through diet), can reflect a bird's nutritional status. A diet rich in carotenoids (from fruits, seeds, and plants) allows parrots to display vibrant hues, which may serve as honest signals of their ability to secure high-quality food.\n\n2. **Species Recognition**:\n   - **Social Cohesion**: Parrots are highly social and often live in flocks. Distinctive coloration helps individuals recognize members of their 

By default, batch() will only return the final output for the entire batch. If you want to receive the output for each individual input as it finishes generating, you can stream results with batch_as_completed():

In [122]:
for response in llm.batch_as_completed([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]):
    print(response)

(0, AIMessage(content="Parrots exhibit vibrant, colorful feathers due to a combination of evolutionary, ecological, and social factors. Here's a structured breakdown of the primary reasons:\n\n1. **Sexual Selection and Mate Attraction**:\n   - **Mate Choice**: Bright colors often signal genetic fitness and health. In many parrot species, males with more vivid plumage are preferred by females, as these colors can indicate a robust immune system and access to high-quality food sources.\n   - **Carotenoid Pigments**: The intensity of colors like reds, yellows, and oranges is derived from carotenoids obtained through diet. These pigments are not synthesized by the birds but must be acquired from food, making their presence a reliable indicator of foraging efficiency and overall health.\n\n2. **Social Communication**:\n   - **Species and Individual Recognition**: Parrots live in complex social structures. Colorful plumage helps individuals recognize each other, especially in dense habitats 